If you have any question, you **can ask me through Discord -> diwahsap** (ML-59)


`Catatan`
1. Jangan lupa untuk masukkan email dan password di *Login, to Get Access Token*
2. Upload berkas xlsx, dan ubah pathnya di *Load Data*
3. Tutorial [sini](https://youtu.be/mABOxEzuEtY)
---



In [ ]:
!pip install fuzzywuzzy[speedup]

In [ ]:
import json
import numpy as np
import pandas as pd
from fuzzywuzzy import process
import requests

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

# Login, to Get Access Token

In [ ]:
url = "https://api.kampusmerdeka.kemdikbud.go.id/v1alpha1/mentors/login"

headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Content-Type": "application/json",
    "Origin": "https://mentor.kampusmerdeka.kemdikbud.go.id",
    "Referer": "https://mentor.kampusmerdeka.kemdikbud.go.id/",
    "sec-ch-ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
}

data = {
    "email": "",
    "password": ""
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code)
print(response.json())


In [ ]:
# Extract the access token
access_token = response.json()['data']['access_token']

# Print the access token
print(access_token)

# Get Program ID

In [ ]:
# URL for the GET request
url = 'https://api.kampusmerdeka.kemdikbud.go.id/v1alpha2/mentors/programs'

# Headers for the GET request
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-US,en;q=0.9',
    'Authorization': f'Bearer {access_token}',
    'Origin': 'https://mentor.kampusmerdeka.kemdikbud.go.id',
    'Referer': 'https://mentor.kampusmerdeka.kemdikbud.go.id/',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

# Make the GET request
response = requests.get(url, headers=headers)

# Print the response (or handle it as needed)
print(response.json())

In [ ]:
# Extract the program_id
program_id = response.json()['data'][0]['program_id']

# Print the program_id
print(program_id)

# Get Activity ID

In [ ]:
url = f"https://api.kampusmerdeka.kemdikbud.go.id/v1alpha1/mentors/me/mentees?program_id={program_id}&offset=0&limit=50"

# Make the GET request
response = requests.get(url, headers=headers)

# Print the response (or handle it as needed)
print(response.status_code)
print(response.json())

In [ ]:
# Extract the program_id
activity_id = response.json()["data"][0]["activity_id"]

# Print the program_id
print(activity_id)

# Get Student Name and ID Student

In [ ]:
# Extract just the id and name from each entry in the data list
id_name_list = [{'id': item['id'], 'name': item['name']} for item in response.json()['data']]

# Print the extracted list
print(id_name_list)

# Get ID Module and Module Name

In [ ]:
url = f"https://api.kampusmerdeka.kemdikbud.go.id/v1alpha1/mentors/me/mentees/{id_name_list[0]['id']}/activities/{activity_id}/final_assessment"

# Make the GET request
response = requests.get(url, headers=headers)
response_data = response.json()

# Print the response (or handle it as needed)
print(response.status_code)
print(response.json())

# Load Data

In [ ]:
# Load the spreadsheet
file_path = '/content/ML-59 - Final Evaluation Bangkit 2024 Batch 1.xlsx'
spreadsheet = pd.ExcelFile(file_path)

# Load the data from both sheets
sheet1_data = pd.read_excel(spreadsheet, 'Sheet1')
comments_data = pd.read_excel(spreadsheet, sheet_name=1)

# Data from the first sheet
students_scores = sheet1_data.set_index('Nama')

# Comments for different score ranges from the second sheet
comments_data = comments_data.set_index('Course List')

# Function to get the comment based on the module name and score
def get_comment(module_name, score):
    closest_module_name = process.extractOne(module_name, comments_data.index.tolist())[0]
    if score >= 70:
        return comments_data.loc[closest_module_name, 'Comment for score range : 100 - 70']
    elif score >= 40:
        return comments_data.loc[closest_module_name, 'Comment for score range : 60 - 40']
    else:
        return comments_data.loc[closest_module_name, 'Comment for score range : 30-0']


def generate_student_json(response_data, student_name):
    # Fuzzy match the student name
    closest_student_name = process.extractOne(student_name, students_scores.index.tolist())[0]

    # Transform the data to fit the required template
    result = {
        "comment": response_data['data']['comment'],
        "status": response_data['data']['status'],
        "scores": []
    }

    # Iterate through the modules and construct the score data
    for module in response_data['data']['modules']:
        module_id = module['id']
        module_name = module['name']

        # Fuzzy match the module name
        closest_module_name = process.extractOne(module_name, students_scores.columns.tolist())[0]

        score = students_scores.loc[closest_student_name, closest_module_name]
        comment = get_comment(module_name, score)

        result["scores"].append({
            "module_id": module_id,
            "module_name": closest_module_name,
            "skill_id": "",
            "skill_name": "",
            "comment": comment,
            "score": score
        })

    json_data = json.dumps(result, cls=NpEncoder)
    return json_data

# Send Data All Students

In [ ]:
for item in id_name_list:
    id = item['id']
    name = item['name']

    print(f"Processing, {id}, {name}")

    url = f"https://api.kampusmerdeka.kemdikbud.go.id/v1alpha1/mentors/me/mentees/{id}/activities/{activity_id}/final_assessment"
    payload = generate_student_json(response_data, name)
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.status_code, "-----\n")

    # save payload into file
    with open(f"{name}.json", "w") as f:
        f.write(payload)